In [36]:
from dice import roll
import gym
from gym.spaces import Dict, Box, Discrete
import numpy as np
from random import random
from copy import copy

In [111]:
class BackgammonEnv(gym.Env):
    def __init__(self):
  
        # Tesauro's backgammon algorithms encode the board using a specific 'truncated unary encoding' with some quirks. You can read more about it here:
        # https://en.wikipedia.org/wiki/Unary_coding 
        # http://www.scholarpedia.org/article/User:Gerald_Tesauro/Proposed/Td-gammon
        # https://www.bkgm.com/rgb/rgb.cgi?view+610

        # although its purported benefits are not immediately apparent to us (perhaps we've been spoiled by the advances in computational power which have made such optimizations optional?),
        # our encoding is largely based on the one used in Tesauro's 'pubeval.c', with the only changes made being the order of the values, and perhaps the use of two such figurative boards 
        # (one for each player) rather than one 

        # defining some ranges for our observation space
        low = np.zeros((97,)) # 1 value for the bar + 4 per each of the 24 positions on the board
        high = np.ones((96,)) # the first three of the four values encoding the number of checkers at a given position are either 0 or 1...

        for i in range(3, 97, 4): # but every fourth value can go as high as (15-3)/2 = 6...
            high[i] = 6.0

        high = np.insert(arr=high,obj=0,values=7.5) # and the very first value, which encodes the number of checkers on the bar, can go as high as 15/2 = 7.5
        # to make them more readable to humans, our observations are dictionaries
        # but we can't feed dictionaries to our ANN, so we are going to have to use gym's 'FlattenObservation' wrapper to flatten the dictionary into a single array later
        self.observation_space = Dict(
            {
                # player 'White'
                'W': Dict(
                    {
                        'board': Box(low=low, high=high,dtype=np.float32), # the board, consisting of the bar and 24 'points'
                        'menoff': Box(low=0.0,high=1.0,shape=(1,),dtype=np.float32), # number of checkers removed from the board as a fraction of the total number of checkers i.e. n/15
                        'turn': Discrete(2) # '1' if it is this player's turn, '0' if not
                    }
                ),
                # player 'Black'
                'B': Dict(
                    {
                        'board': Box(low=low, high=high,dtype=np.float32),
                        'menoff': Box(low=0.0,high=1.0,shape=(1,),dtype=np.float32),
                        'turn': Discrete(2)
                    }
                )
            }
        )
        
        # which actions can be taken at any given time step depend on the state of the board and the roll of the dice. hence we choose to not attempt to define the action space
        self.action_space = None
        
        # create an empty board:
        # to allow for indexing of board positions, the environment's state uses an array of arrays to store the values of the bar and each point
        # later we simply flatten this array to create observations

        self.state = {
            'W': {
                'board': np.zeros((24, 4)),
                'barmen': 0,
                'menoff': 0,
                'turn': 0
            },
            'B': {
                'board': np.zeros((24, 4)),
                'barmen': 0,
                'menoff': 0,
                'turn': 0
            }
        }

        # the previously mentioned truncated unary encoding:
        self.encoding = {
            0: np.array([0.,0.,0.,0.]),
            1: np.array([1.,0.,0.,0.]),
            2: np.array([1.,1.,0.,0.]),
            3: np.array([1.,1.,1.,0.]),
            4: np.array([1.,1.,1.,0.5]),
            5: np.array([1.,1.,1.,1.]),
            6: np.array([1.,1.,1.,1.5]),
            7: np.array([1.,1.,1.,2.0]),
            8: np.array([1.,1.,1.,2.5]),
            9: np.array([1.,1.,1.,3.0]),
            10: np.array([1.,1.,1.,3.5]),
            11: np.array([1.,1.,1.,4.0]),
            12: np.array([1.,1.,1.,4.5]),
            13: np.array([1.,1.,1.,5.0]),
            14: np.array([1.,1.,1.,5.5]),
            15: np.array([1.,1.,1.,6.0])
        }

        # define the game's starting position:
  
        # create an empty board without a bar
        self.starting_pos = np.zeros((24, 4))

        # place the correct number of checkers in the correct positions
        self.starting_pos[0] = copy(self.encoding[2])
        self.starting_pos[11] = copy(self.encoding[5])
        self.starting_pos[16] = copy(self.encoding[3])
        self.starting_pos[18] = copy(self.encoding[5])

    def _get_obs(self, obs):
        w_board = obs['W']['board']
        w_board = w_board.flatten()
        
        b_board = obs['B']['board']
        b_board = b_board.flatten()

        observation = []
        observation = np.append(observation,w_board)
        observation = np.append(observation,obs['W']['barmen'])
        observation = np.append(observation,obs['W']['menoff'])
        observation = np.append(observation,obs['W']['turn'])
        observation = np.append(observation,b_board)
        observation = np.append(observation,obs['B']['barmen'])
        observation = np.append(observation,obs['B']['menoff'])
        observation = np.append(observation,obs['B']['turn'])
        return observation

    def _get_info(self):
        pass

    def reset(self): 

        # a 'coin flip' to determine which side goes first
        coin = int(random()>0.5)

        # reset the board to the game's starting position and assign a turn order based on the coin flip
        self.state = {
            'W': {
                'board':copy(self.starting_pos), 
                'barmen': 0,
                'menoff': 0,
                'turn': coin
            },
            'B': {
                'board':copy(self.starting_pos),
                'barmen': 0,
                'menoff': 0,
                'turn': 1-coin
            }
        }

        return self.state

    def step(self, action):

        # who's turn is it?
        if self.state['W']['turn'] == 1:
            player = 'W'
            opponent = 'B'
        else:
            player = 'B'
            opponent = 'W'

        # assume for now that an action is a list of up to four 'old position - new position' tupels
        # move = (int,int)
        # action  = [move,move]
        for move in action:

            # 'LIFTING' A CHECKER

            old_pos, new_pos = move
            # are we moving a piece off the bar?
            if old_pos == -1:
                # remove a checker from the bar
                self.state[player]['barmen'] -= 0.5

            else:
                # get the current number of checkers at the position from which we need to remove a checker
                encoded_checkers = self.state[player]['board'][old_pos]
                print(encoded_checkers)
                # decode
                for key, value in self.encoding.items():
                    if np.array_equal(encoded_checkers,value):
                        n_checkers = key
                # subtract a checker
                self.state[player]['board'][old_pos] = copy(self.encoding[n_checkers-1])

            # 'PLACING DOWN' A CHECKER

            # are we bearing off?
            if new_pos == 25:
                self.state[player]['menoff'] += 1/15

            else:
                # get the current number of checkers at the position to which we need to add a checker
                encoded_checkers = self.state[player]['board'][new_pos]
                # decode
                for key, value in self.encoding.items():
                    if np.array_equal(encoded_checkers,value):
                        n_checkers = key
                # add a checker
                self.state[player]['board'][new_pos] = copy(self.encoding[n_checkers+1])

                # check for blots
                mirror_pos = new_pos+25-2*new_pos-1
                if not np.array_equal(self.state[opponent]['board'][mirror_pos],[0,0,0,0]):
                    # if there is a blot, move the opponent's piece to the bar
                    self.state[opponent]['board'][mirror_pos] = [0,0,0,0]
                    self.state[opponent]['barmen'] += 0.5

            # 0 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
            #  24 23 22 21 20 19 18 17 16 15 14 13 12 11 10  9  8  7  6  5  4  3  2  1 0

        # swap whose turn it is
        self.state['W']['turn'] = 1-self.state['W']['turn']
        self.state['B']['turn'] = 1-self.state['B']['turn']

        # !!! nicht mehr hier flatten sondern stadtdessen n der neuen Funktion !!!
        # observation = self._get_obs()

        # reward is zero unless one of four conditions is met:
        reward = 0

        # 1) White wins, Black is gammoned
        if self.state['W']['menoff'] ==  1 and self.state['B']['menoff'] == 0:
            reward = 2
        # 2) White wins
        elif self.state['W']['menoff'] ==  1:
            reward = 1
        # 3) Black wins, White is gammoned
        elif self.state['B']['menoff'] ==  1 and self.state['W']['menoff'] == 0: 
            reward = -2
        # 4) Black wins
        elif self.state['B']['menoff'] ==  1:
            reward = -1

        # if one of the four conditions is met, the game is finished and the episode ends
        done = reward != 0

        return self.state, reward, done
    
    def get_state(self, obsevation, action):
        
        # who's turn is it?
        if observation['W']['turn'] == 1:
            player = 'W'
            opponent = 'B'
        else:
            player = 'B'
            opponent = 'W'

        # assume for now that an action is a list of up to four 'old position - new position' tupels
        # move = (int,int)
        # action  = [move,move]
        for move in action:

            # 'LIFTING' A CHECKER

            old_pos, new_pos = move
            # are we moving a piece off the bar?
            if old_pos == -1:
                # remove a checker from the bar
                observation[player]['barmen'] -= 0.5

            else:
                # get the current number of checkers at the position from which we need to remove a checker
                encoded_checkers = observation[player]['board'][old_pos]
                print(encoded_checkers)
                # decode
                for key, value in self.encoding.items():
                    if np.array_equal(encoded_checkers,value):
                        n_checkers = key
                # subtract a checker
                observation[player]['board'][old_pos] = copy(self.encoding[n_checkers-1])

            # 'PLACING DOWN' A CHECKER

            # are we bearing off?
            if new_pos == 24:
                observation[player]['menoff'] += 1/15

            else:
                # get the current number of checkers at the position to which we need to add a checker
                encoded_checkers = observation[player]['board'][new_pos]
                # decode
                for key, value in self.encoding.items():
                    if np.array_equal(encoded_checkers,value):
                        n_checkers = key
                # add a checker
                observation[player]['board'][new_pos] = copy(self.encoding[n_checkers+1])

                # check for blots
                mirror_pos = new_pos+25-2*new_pos-1
                if not np.array_equal(observation[opponent]['board'][mirror_pos],[0,0,0,0]):
                    # if there is a blot, move the opponent's piece to the bar
                    observation[opponent]['board'][mirror_pos] = [0,0,0,0]
                    observation[opponent]['barmen'] += 0.5

            # 0 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
            #  24 23 22 21 20 19 18 17 16 15 14 13 12 11 10  9  8  7  6  5  4  3  2  1 0

        # swap whose turn it is
        observation['W']['turn'] = 1-observation['W']['turn']
        observation['B']['turn'] = 1-observation['B']['turn']
        
        return self._get_obs(observation)

        
 # W:    0 1 ... 24
# B:      24 ... 1 0

# translating from one board to the other: n+25-(2n)

# idea: replace W and B with 1 and 0 

In [118]:
class Agent:
    def __init__(self, env):
        self.env = env
        #self.gamma = gamma
        #self.network = network

    def choose_action(self, obs):
        
        dice = roll()
        
        legal_actions = []
        size = len(dice)
        action = []

        # check whose turn it is
        if obs['W']['turn'] == 1:
            player_obs = obs['W'].copy()
            opponent_obs = obs['B']
        else:
            player_obs = obs['B'].copy()
            opponent_obs = obs['W']
            
        # make board more easily readable
        new_board = []
        for idx, pos in enumerate(player_obs['board']):
            new_board.append(pos[0] + pos[1] + pos[2] + pos[3] * 2)
            if new_board[idx] > 0:
                print(idx)
        player_obs['board'] = new_board
        
        # recursive function to search "action-tree"
        def find_board_actions(action, dice, player_obs):
            # if our action has reached the desired size it is appended and we return
            # !!! problem: what if we cannot use all the dice?? We still need to find legal actions !!!
            if len(action) == size:
                legal_actions.append(action)
                return

            # in case there are chips in the bar they have to be removed before any other actions can be taken
            if player_obs['barmen'] > 0:
                for i, die in enumerate(dice):
                    if opponent_obs['board'][die + 25 - 2 * die - 1][1] == 0:
                        new_player_obs = player_obs.copy()
                        new_player_obs['board'][die] += 1
                        new_player_obs['barmen'] -= 1
                        find_board_actions(action.copy() + [(-1, die)], dice[1:], new_player_obs)

            # is it legal to move off the board?
            moveoff = True
            for idx in range(18):
                if player_obs['board'][idx] > 1:
                    moveoff = False
                    break

            for idx, pos in enumerate(player_obs['board']):
                if pos > 0 and (idx + dice[0]) > 23:
                    if moveoff:
                        new_player_obs = player_obs.copy()
                        new_player_obs['board'][idx] -= 1
                        find_board_actions(action.copy() + [(idx + 1, 25)], dice[1:], new_player_obs)

                elif pos > 0 and opponent_obs['board'][(idx + dice[0]) + 25 - 2 * (idx + dice[0]) - 1][1] == 0:
                    new_player_obs = player_obs.copy()
                    new_player_obs['board'][idx] -= 1
                    print(idx)
                    new_player_obs['board'][idx + dice[0]] += 1
                    find_board_actions(action.copy() + [(idx, idx + dice[0])], dice[1:], new_player_obs)

        find_board_actions(action, dice, player_obs)
        
        # test print out looking good for initial turn
        print(legal_actions)
        
        states = [] 
        # call function that returns the state 
        # maybe a flattened version of the state that can immediately be fed into the network??
        print(obs)
        for action in legal_actions:
            states.append(env.get_state(obs.copy(), action))
        
        #values = []
        #for state in states:
        #    values.append(self.network.call(state))
            
        # !!! missing something that evaluates at which value index we have the best probability of winning !!!
        # maybe using numpy arrays??
        
        #return legal_actions[index]
        print(legal_actions[0])
        return legal_actions[0]

    def learn(self):
        pass

In [119]:
env = BackgammonEnv()
agent = Agent(env)
observation = env.reset()

In [120]:
for i in range(10):
    action = agent.choose_action(observation)
    observation, reward, done = env.step(action)
    print(observation)
    

0
11
16
18
0
0
11
15
16
18
19
2
11
15
16
18
19
16
11
15
18
19
18
11
15
18
19
20
11
15
18
19
[[(0, 2), (0, 4)], [(0, 2), (11, 15)], [(0, 2), (15, 19)], [(0, 2), (16, 20)], [(0, 2), (18, 22)], [(0, 2), (19, 23)], [(2, 4), (11, 15)], [(2, 4), (15, 19)], [(2, 4), (16, 20)], [(2, 4), (18, 22)], [(2, 4), (19, 23)], [(16, 18), (11, 15)], [(16, 18), (15, 19)], [(16, 18), (18, 22)], [(16, 18), (19, 23)], [(18, 20), (11, 15)], [(18, 20), (15, 19)], [(18, 20), (18, 22)], [(18, 20), (19, 23)], [(20, 22), (11, 15)], [(20, 22), (15, 19)], [(20, 22), (18, 22)], [(20, 22), (19, 23)]]
{'W': {'board': array([[1., 1., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [1., 1., 1., 1.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [1

KeyError: -1